# Parameter Sweep
Last Edit: 07.19.2020 CYZ
This code perfoms a parameter sweep

In [1]:
import numpy as np
import pandas as pd
from scipy import integrate

from utils.seir_model import seir_model_shields_rcfc_nolatent_time

# Load Default Parms


# Parameters

In [2]:
# Obtain defaults by executing 'set_default_parms.py' script
exec(open('utils/set_default_parms.py').read())

# returns: pars, epiPars, contactPars, intvPars, inits, X0

# Run Baseline Model

In [3]:
# Model
t0 = 0
tf = pars['nDays']
tstep = 1 # day
t = np.arange(t0,tf+tstep, tstep)
combPars = list((pars, epiPars, contactPars, intvPars))


# Run Model
model_out = integrate.odeint(seir_model_shields_rcfc_nolatent_time
                             , X0
                             , t
                             , args=(combPars,))#, rtol = 1, hmax=0.1, )

/mnt/c/Users/czhao/Documents/CYZ Github/Weitz Group/COVID-19/Lopman COVID collab/Serological_Shielding/Python/utils/seir_model.py:107: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  CM_School_Comb_reopen[temp_even_10x10_Indices] = contactPars['SchoolContacts_5x5'].T # meshgrid transposed


# Run Parameter Sweep

In [4]:
# Parameter Sweep
cvals = np.arange(3)/2
    #cvals = np.arange(9)/8
    #cvals = np.arange(5)/4
    
test_rates = np.append(np.array([0]), np.logspace(3,7, 5))
    #test_rates = np.append(np.array([0]), np.logspace(1,7, 13))

alphas = np.array([4])
    #alphas = np.array([0, 0.5, 1, 2, 4, 10, 20])

specificities = np.array([0.96])
    #specificities = np.array([0.5, 0.75, 0.85, 0.90, 0.96, 1])

sensitivities = np.array([0.94])
    #sensitivities = np.array([0.5, 0.94])

sweep = np.array([(i_c, i_test, i_alpha, i_spec, i_sens) for i_c in cvals for i_test in test_rates for i_alpha in alphas for i_spec in specificities for i_sens in sensitivities])

In [5]:
df_sweep = np.zeros((sweep.shape[0], 9))
for i in np.arange(sweep.shape[0]):
    print(i/sweep.shape[0]*100)
    
    # Load ins
    c_in, test_in, alpha_in, spec_in, sens_in = sweep[i,:]
    
    # Save to combPars
    intvPars['c'] = c_in
    intvPars['daily_tests'] = test_in
    intvPars['alpha'] = alpha_in
    intvPars['specificity'] = spec_in
    intvPars['sensitivity'] = sens_in
    combPars = list((pars, epiPars, contactPars, intvPars))
    
    # Run Model
    model_out = integrate.odeint(seir_model_shields_rcfc_nolatent_time, X0, t, args=(combPars,))#, rtol = 1, hmax=0.1, )

    # Stats
    tot_deaths = np.sum(model_out[-1, pars['D_ids']])
    
    tot_infec = np.sum(model_out[-1,pars['Isym_ids']]
                     + model_out[-1,pars['Iasym_ids']]
                     + model_out[-1,pars['Hsub_ids']]
                     + model_out[-1,pars['Hcri_ids']]
                     + model_out[-1,pars['D_ids']]
                     + model_out[-1,pars['R_ids']]
                     + model_out[-1,pars['Isym_pos_ids']]
                     + model_out[-1,pars['Iasym_pos_ids']]
                     + model_out[-1,pars['R_pos_ids']])
    
    tot_cases = np.sum(model_out[-1,pars['Hsub_ids']]
                     + model_out[-1,pars['Hcri_ids']]
                     + model_out[-1,pars['D_ids']]
                     + model_out[-1,pars['Isym_pos_ids']]
                     + model_out[-1,pars['Iasym_pos_ids']]
                     + model_out[-1,pars['R_pos_ids']])
    
    tot_released = np.sum(model_out[-1,pars['S_pos_ids']]
                      + model_out[-1,pars['E_pos_ids']]
                      + model_out[-1,pars['Isym_pos_ids']]
                      + model_out[-1,pars['Iasym_pos_ids']]
                      + model_out[-1,pars['R_pos_ids']])
    
    df_sweep[i,:] = [tot_deaths, tot_infec, tot_cases, tot_released, c_in, test_in, alpha_in, spec_in, sens_in]

0.0
5.555555555555555
11.11111111111111
16.666666666666664
22.22222222222222
27.77777777777778
33.33333333333333
38.88888888888889
44.44444444444444
50.0
55.55555555555556
61.111111111111114
66.66666666666666
72.22222222222221
77.77777777777779
83.33333333333334
88.88888888888889
94.44444444444444


In [6]:
# Save Result
df_out = pd.DataFrame(df_sweep)
df_out.columns = ['Deaths', 'Infected', 'Known_Infection', 'Released', 'c', 'test_rate', 'alpha', 'specificity', 'sensitivity']
df_out.to_csv('04_24_2020_ParmSweep_Python.csv')